**3.特征工程**

从用户表、商品表、评论表、2月用户行为数据、3月用户行为数据、4月用户行为数据六个原始数据集中共提取293个数据特征，最终构建训练集、验证集和测试集。下面分11个部分进行讲解：

首先展示最终得到的训练集：

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 6)
training_set = pd.read_csv('data/result_训练集.csv',engine='python',encoding = 'utf-8')
training_set 

,用户ID,商品ID,商品类别,3天内商品-行为总数_1.0,3天内商品-行为总数_2.0,3天内商品-行为总数_3.0,3天内商品-行为总数_4.0,3天内商品-行为总数_5.0,3天内商品-行为总数_6.0,3天内商品类别-行为总数_1.0,...,商品类别-收藏行为均值,商品类别-点击行为均值,存在差评,差评率,评论数量_0,评论数量_1,评论数量_2,评论数量_3,评论数量_4,标签
0,202633.0,12564.0,8.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,...,20.866667,5167.4,1.0,0.0260,0.0,0.0,0.0,0.0,1.0,1.0
1,218498.0,149854.0,8.0,4.0,0.0,0.0,0.0,0.0,4.0,6.0,...,20.866667,5167.4,1.0,0.0403,0.0,0.0,0.0,0.0,1.0,1.0
2,221842.0,75877.0,8.0,3.0,0.0,0.0,0.0,0.0,5.0,82.0,...,20.866667,5167.4,1.0,0.0245,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20842,236656.0,59869.0,9.0,4.0,0.0,0.0,0.0,0.0,6.0,8.0,...,46.866667,10623.1,1.0,0.0238,0.0,0.0,0.0,1.0,0.0,0.0
20843,255300.0,93859.0,7.0,8.0,0.0,0.0,0.0,0.0,14.0,32.0,...,56.166667,15101.5,0.0,0.0000,0.0,0.0,0.0,0.0,1.0,0.0
20844,255216.0,81090.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,126.0,...,143.900000,41669.1,1.0,0.0594,0.0,0.0,0.0,0.0,1.0,0.0


然后展示以下11个部分：

**3.1.数据导入与内存占用空间压缩**

导入所需库

In [2]:
import time
from datetime import datetime
from datetime import timedelta
import pandas as pd
import pickle 
import os
import math
import numpy as np

将读取的数据从64位转化为32为，降低内存的占用量。

In [3]:
#以2月行为数据为例，读取数据
test = pd.read_csv('data/data_2月用户行为数据.csv',engine='python',encoding = 'utf-8')

#转换数据类型，变成32位，减少内存占用量
test[['用户ID','商品ID','行为类别','商品类别','商品品牌']] = test[['用户ID','商品ID','行为类别','商品类别','商品品牌']].astype('float32')

test.dtypes#打印类型
test.info()#打印文件大小

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11485384 entries, 0 to 11485383
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   用户ID    float32
 1   商品ID    float32
 2   时间      object 
 3   行为类别    float32
 4   商品类别    float32
 5   商品品牌    float32
dtypes: float32(5), object(1)
memory usage: 306.7+ MB


In [4]:
#以2月行为数据为例，读取数据
test = pd.read_csv('data/data_2月用户行为数据.csv',engine='python',encoding = 'utf-8')

test.dtypes#打印类型
test.info()#打印文件大小

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11485384 entries, 0 to 11485383
Data columns (total 6 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   用户ID    int64 
 1   商品ID    int64 
 2   时间      object
 3   行为类别    int64 
 4   商品类别    int64 
 5   商品品牌    int64 
dtypes: int64(5), object(1)
memory usage: 525.8+ MB


上诉结果解释：以32位的读取方式读取，内存占用300+MB；以64位的读取方式读取，内存占用500+MB。对比内存占用量可以得出，以32位读取方式读入数据更好，减少内存的占用量。所以，一下均采用32位的读取方式，读取数据。

数据文件的宏定义

In [5]:
action1_path = r'data/data_2月用户行为数据.csv'
action2_path = r'data/data_3月用户行为数据.csv'
action3_path = r'data/data_4月用户行为数据.csv'
comment_path = r'data/data_评论表.csv'
product_path = r'data/data_商品表.csv'
user_path = r'data/data_用户表.csv'

分别读取三个月的数据，然后转成32位，把三个表连在一起

In [6]:
#定义get_action函数，设置以32位分块读取数据的方式
def get_action(file_path):
    
    #读取数据
    #指定参数iterator=True返回一个可迭代对象TextfileReader，以便逐块处理文件， chunksize = 5000每次读取5000条数据
    reader = pd.read_csv(file_path, iterator=True, chunksize = 5000,engine='python',encoding = 'utf-8')
    chunks = []
    for chunk in reader:
        #转换数据类型，换成32位，减小内存占用量
        chunk[['用户ID','商品ID','行为类别','商品类别','商品品牌']] = chunk[['用户ID','商品ID','行为类别','商品类别','商品品牌']].astype('float32')
        chunks.append(chunk)
    
    action = pd.concat(chunks, ignore_index=True)     
    return action
#定义get_all_action函数读取2、3、4月用户行为数据并合并
def get_all_action():
    #调用get_action函数，采取分块读取数据的方式读取2、3、4月数据
    action1 = get_action(action1_path)
    action2 = get_action(action2_path) 
    action3 = get_action(action3_path)
    actions = pd.concat([action1, action2, action3])
    return actions

#定义get_actions函数，读取起始时间=终止时间片段内的数据
def get_actions(start_date, end_date, all_actions):
    """
    :param start_date:
    :param end_date:
    :return: actions: pd.Dataframe
    """
    actions = all_actions[(all_actions.时间>= start_date) & (all_actions.时间 < end_date)].copy()
    return actions

获取合并后2、3、4月用户行为数据集

In [ ]:
all_actions = get_all_action()#调用 get_all_action函数，读取并合并2、3、4月用户行为数据集

In [ ]:
all_actions

In [ ]:
all_actions.info()

In [ ]:
all_actions['时间']=all_actions['时间'].astype('datetime64')

In [ ]:
all_actions.info()

**3.2.用户、商品、评论列的特征提取（热编码）**

**3.2.1.提取用户基本特征**

基于用户本身属性多为类别特征的特点，对年龄、性别、用户级别进行独热编码操作，对于用户注册时间暂时不处理。

3.2.1.1.原始数据如下

In [ ]:
user_base_info = pd.read_csv(user_path, engine='python', encoding='utf-8')
user_base_info.head() 

3.2.1.2.实现函数如下：

In [ ]:
from sklearn import preprocessing
#定义get_user_feature函数，构建用户基本特征表
def get_user_feature(user_base_info):
    
    #对缺失值处理，how='any'：any表示有任何NA存在就删除所在行或列，True表示在原表上修改
    user_base_info.dropna(axis=0 , how='any',inplace=True)
    
    #数值类型转换
    user_base_info['性别'] = user_base_info['性别'].astype(int)    
    user_base_info['年龄'] = user_base_info['年龄'].astype(int) 
    
    model_lableEncoder = preprocessing.LabelEncoder()#标准化标签，将标签值统一转换成range（标签值个数-1）范围内
    #对“年龄”列的原始数据进行编码，将“年龄”列的原属性值“-1”，“1”，“2”，“3”，“4”，“5”，“6”依次编码为 0，1,2,3,4,5,6
    #年龄为1代表15岁以下，2代表26-35岁，3代表36-45岁，4代表46-55岁，5代表56-65岁，6代表66岁以上，-1代表缺失值。
    age = model_lableEncoder.fit_transform(user_base_info['年龄'])
    
    #做一个独热编码，指定编码前缀名称：“年龄”，“性别”，“用户级别”
    age = pd.get_dummies(age, prefix='年龄')
    sex = pd.get_dummies(user_base_info['性别'], prefix='性别')
    user_level = pd.get_dummies(user_base_info['用户级别'], prefix='用户级别')
    
    #将特征按列连接
    user_feature = pd.concat([user_base_info['用户ID'], age, sex, user_level], axis=1)
    return user_feature

3.2.1.3最终处理结果如下：展示最终得到的用户基本特征表

In [ ]:
get_user_feature(user_base_info)#调用get_user_feature函数，提取用户基本特征

**3.2.2.提取商品基本特征**

根据商品表提取商品基本特征，对属性1、属性2、属性3进行独热编码，商品类别和品牌直接作为特征

3.2.2.1.原始数据如下：

In [ ]:
product = pd.read_csv(product_path, engine='python', encoding='utf-8')
product.head()

3.2.2.2.实现函数如下：

In [ ]:
#定义get_product_feature函数，构建商品基本特征表
def get_product_feature(product):
    
    #对商品属性1、属性2、属性3做独热编码
    dummies_attribute1 = pd.get_dummies(product["属性1"], prefix="属性1")
    dummies_attribute2 = pd.get_dummies(product["属性2"], prefix="属性2")
    dummies_attribute3 = pd.get_dummies(product["属性3"], prefix="属性3")
    
    #将各特征列连接
    product = pd.concat([product[['商品ID', '商品类别', '商品品牌']], dummies_attribute1, dummies_attribute2, dummies_attribute3], axis=1)
    return product

3.2.2.3.最终处理结果如下：展示最终得到的商品基本特征表

In [ ]:
get_product_feature(product)#调用get_product_feature函数，提取商品基本特征

**3.2.3.提取评论特征**

分时间段统计数据，并对“评论数量”；列进行独热编码，将“差评率”和“存在差评”两列保存，直接作为特征

3.2.3.1.原始数据如下：

In [ ]:
comments = pd.read_csv(comment_path, engine='python', encoding='utf-8') 
comments

3.2.3.2.实现函数如下

In [ ]:
#设置起始时间
comment_date = [
    "2016-02-01", "2016-02-08", "2016-02-15", "2016-02-22", "2016-02-29",
    "2016-03-07", "2016-03-14", "2016-03-21", "2016-03-28", "2016-04-04",
    "2016-04-11", "2016-04-15"
]

#定义get_comments_feature函数，构建评论特征表
def get_comments_feature(end_date):
    comments = pd.read_csv(comment_path, engine='python', encoding='utf-8')#读取评论表
    comment_date_end = end_date#将传入的end_date时间赋值给comment_date_end
    comment_date_begin = comment_date[0]#将上诉所给时间片段的第一个时间值“2016-02-01”赋值给comment_date_begin
    #找到给定起始时间comment_date，距离end_date之前最近的时间
    for date in reversed(comment_date):
        if date < comment_date_end:
            comment_date_begin = date
            break
    comments = comments[comments.评论时间 == comment_date_begin] 
    #对评论数量进行独热编码
    dummies_comments= pd.get_dummies(comments['评论数量'], prefix='评论数量')
    #某个时间段没有评论是，设置为0
    for i in range(0, 5):
        if '评论数量_' + str(i) not in dummies_comments.columns:
            dummies_comments['评论数量_' + str(i)] = 0
    dummies_comments = dummies_comments[['评论数量_0', '评论数量_1', '评论数量_2', '评论数量_3', '评论数量_4']]
    
    #将各特征按列连接
    comments = pd.concat([comments, dummies_comments], axis=1)
    comments = comments[['商品ID', '存在差评', '差评率','评论数量_0', '评论数量_1', 
                         '评论数量_2', '评论数量_3', '评论数量_4']]
    return comments

3.2.3.3.最终处理结果如下：展示最终得到的评论特征表

In [ ]:
#在此仅给定终止时间为2016年2月4日，以距离展示最终得到的评论特征表
get_comments_feature('2016-02-04')#调用get_comments_feature函数，提取评论特征

**3.3.用户ID-商品ID-行为总数的特征提取**

统计某一段时间内，每一个用户对每一个目标商品执行不同操作的数量、每一个用户对每一类目标商品执行不同操作的数量、每一个用户对不同商品类别中除目标商品外的其他商品执行不同操作的数量、不同类别下目标商品的行为数量与该时间段目标商品的行为数量均值的差值。

3.3.1.实现函数如下：

In [ ]:
#定义get_action_feature函数，完成行为特征表的构建
def get_action_feature(start_date, end_date, all_actions, i):
    
    #定义列名称的字符串
    before_date1 = '%s天内商品-行为总数' %i
    before_date2 = '%s天内商品类别-行为总数' %i
    before_date3 = '%s天内其他商品-行为总数' %i
    before_date4 = '%s天内商品-行为总数均值差' %i
    
    #统计某个用户对某类商品在i天内不同行为总数，得到用户-商品-行为总数表
    actions = get_actions(start_date, end_date, all_actions) #获取起始时间-终止时间片段内的所有数据
    actions_3columns = actions[['用户ID', '商品ID', '商品类别']]#获取用户ID、商品ID、商品类别这3个字段
    actions_1column_dummies = pd.get_dummies(actions['行为类别'],prefix=before_date1 )#对“行为类别”做独热编码，指定前缀，设置长达i天的时间片段
    user_product_actionClass = pd.concat([actions_3columns, actions_1column_dummies], axis=1)#将各特征列连接
    user_product_actionClass = user_product_actionClass.groupby(['用户ID', '商品ID','商品类别'], as_index=False).sum()#统计某个用户对某个商品在i天内不同行为总数
    
    #统计某个用户对某类商品在i天内不同行为总数，得到用户-商品类别-行为总数表
    actions = get_actions(start_date, end_date, all_actions) #获取起始时间-终止时间片段内的所有数据
    actions_2columns = actions[['用户ID',  '商品类别']]#获取用户ID、商品类别2个字段
    actions_1column_dummies = pd.get_dummies(actions['行为类别'], prefix=before_date2)#独热编码，指定前缀，设置长达i天的时间片段
    user_productClass_actionClass = pd.concat([actions_2columns, actions_1column_dummies], axis=1)#将各特征列按列连接
    user_productClass_actionClass = user_productClass_actionClass.groupby(['用户ID', '商品类别'], as_index=False).sum()#统计某个用户对某个商品类别在i天内不同行为总数
    
    action_feature = pd.merge(user_product_actionClass, user_productClass_actionClass, how='left', on=['用户ID','商品类别'])#将上诉得到的两个表连接
    
    #统计同一类别下其他商品行为总数
    action_feature[before_date3+'_1.0'] = action_feature[before_date2+'_1.0'] - action_feature[before_date1+'_1.0']
    action_feature[before_date3+'_2.0'] = action_feature[before_date2+'_2.0'] - action_feature[before_date1+'_2.0']
    action_feature[before_date3+'_3.0'] = action_feature[before_date2+'_3.0'] - action_feature[before_date1+'_3.0']
    action_feature[before_date3+'_4.0'] = action_feature[before_date2+'_4.0'] - action_feature[before_date1+'_4.0']
    action_feature[before_date3+'_5.0'] = action_feature[before_date2+'_5.0'] - action_feature[before_date1+'_5.0']
    action_feature[before_date3+'_6.0'] = action_feature[before_date2+'_6.0'] - action_feature[before_date1+'_6.0']
    
    #按照不同的行为类别，将i天内用户对商品行为总数与每天的商品行为总数作差，把数据中心化，减小过度拟合的可能性
    action_feature[before_date4+'_1.0'] = action_feature[before_date1+'_1.0'] - (action_feature[before_date1+'_1.0']/i)
    action_feature[before_date4+'_2.0'] = action_feature[before_date1+'_2.0'] - (action_feature[before_date1+'_2.0']/i)
    action_feature[before_date4+'_3.0'] = action_feature[before_date1+'_3.0'] - (action_feature[before_date1+'_3.0']/i)
    action_feature[before_date4+'_4.0'] = action_feature[before_date1+'_4.0'] - (action_feature[before_date1+'_4.0']/i)
    action_feature[before_date4+'_5.0'] = action_feature[before_date1+'_5.0'] - (action_feature[before_date1+'_5.0']/i)
    action_feature[before_date4+'_6.0'] = action_feature[before_date1+'_6.0'] - (action_feature[before_date1+'_6.0']/i)
    
    return action_feature

3.3.3.最终处理结果如下：首先展示最终得到的用户ID-商品ID-行为总数特征表

In [ ]:
#时间片段以三天为例，设置一个长达3天的时间片段的起始时间和终止时间
start_date = '2016-02-01'
end_date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=3)
end_date = end_date.strftime('%Y-%m-%d')
#调用get_action_feature函数，提取用户ID-商品ID-行为总数特征
get_action_feature(start_date, end_date, all_actions, 3)  

3.3.3.分布实现get_action_feature()函数的过程如下：

3.3.3.1.首先展示原数据：

In [ ]:
all_actions#2、3、4月用户行为数据合集

3.3.3.2.设置一个长达3天的时间片段的起始时间和终止时间

In [ ]:
train_start_date = '2016-02-01'
train_start_date = datetime.strptime(train_start_date, '%Y-%m-%d')
train_end_date = train_start_date + timedelta(days=3)
train_end_date = train_end_date.strftime('%Y-%m-%d')

3.3.3.3.统计某个用户对某个商品在3天内不同行为总数

In [ ]:
actions = get_actions(train_start_date, train_end_date, all_actions)#获取起始时间-终止时间片段内的所有数据 
actions_3columns = actions[['用户ID', '商品ID', '商品类别']]
actions_1column_dummies = pd.get_dummies(actions['行为类别'], prefix='3天内商品-行为总数')
user_product_actionClass = pd.concat([actions_3columns, actions_1column_dummies], axis=1)  
user_product_actionClass = user_product_actionClass.groupby(['用户ID', '商品ID','商品类别'], as_index=False).sum()#统计某个用户对某个商品在3天内不同行为总数
user_product_actionClass

3.3.3.4.统计某个用户对某个商品类别在3天内不同行为总数

In [ ]:
#统计某个用户对某个商品类别在3天内不同行为总数
actions = get_actions(train_start_date, train_end_date, all_actions)#获取起始时间-终止时间片段内的所有数据 
actions_2columns = actions[['用户ID',  '商品类别']]
actions_1column_dummies = pd.get_dummies(actions['行为类别'], prefix='3天内商品类别-行为总数')
user_productClass_actionClass = pd.concat([actions_2columns, actions_1column_dummies], axis=1) 
user_productClass_actionClass = user_productClass_actionClass.groupby(['用户ID', '商品类别'], as_index=False).sum()#统计某个用户对某个商品类别在3天内不同行为总数
user_productClass_actionClass

3.3.3.5.将上述得到的两个表使用merge函数连接起来

In [ ]:
user_productAndClass_actionClass = pd.merge(user_product_actionClass, user_productClass_actionClass, how='left', on=['用户ID','商品类别'])
UPCA=user_productAndClass_actionClass
UPCA

3.3.3.6.计算在同一商品类别下，除了目标商品，其他商品行为总数，以用户浏览行为（行为类别=1）为例

In [ ]:
before_date1 = '%s天内商品-行为总数' %3
before_date2 = '%s天内商品类别-行为总数' %3
before_date3 = '%s天内其他商品-行为总数' %3 
UPCA[before_date3+'1.0'] = UPCA[before_date2+'_1.0'] - UPCA[before_date1+'_1.0']#计算其他商品-行为总数，以浏览为例
UPCA

**3.4.用户ID-行为总数的特征提取**

分时间段，统计用户不同行为数量，计算其他行为与购买行为的转化率，以及各行为数量均值。

3.4.1.实现函数如下：

In [ ]:
#定义get_accumulate_user_feature函数，构建用户ID-行为总数特征表
def get_accumulate_user_feature(end_date, all_actions, day):
    
    #设置时间片段的其实时间
    start_date = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=day)
    start_date = start_date.strftime('%Y-%m-%d')
    
    before_date = '%s天行为总数' % day
    before_date1 = '%s天' % day
    before_date1  
    
    #统计用户3天内不同行为总数
    actions = get_actions(start_date, end_date, all_actions)#获取起始时间-终止时间片段内的所有数据
    dummies_acionClass = pd.get_dummies(actions['行为类别'], prefix=before_date)#独热编码  
    actions['日期'] = pd.to_datetime(actions['时间']).apply(lambda x: x.date())#转换时间格式
    user_actionClass = pd.concat([actions[['用户ID', '日期']],  dummies_acionClass], axis=1)#将各特征按列连接
    user_actionClass = user_actionClass.groupby(['用户ID'], as_index=False).sum()#统计用户不同行为总数
    
    #计算转化率，统计用户各项行为-购买行为的转化率
    #使用np.log.做减法，相当于做除法，避免除数为0出现空值，得到的结果就是购买/浏览的比重
    user_actionClass[before_date1 + '购买/浏览转化率'] =  np.log(1 + user_actionClass[before_date + '_4.0']) - np.log(1 + user_actionClass[before_date +'_1.0'])
    user_actionClass[before_date1 + '购买/加购转化率'] =  np.log(1 + user_actionClass[before_date + '_4.0']) - np.log(1 + user_actionClass[before_date +'_2.0'])
    user_actionClass[before_date1 + '购买/删购转化率'] =  np.log(1 + user_actionClass[before_date + '_4.0']) - np.log(1 + user_actionClass[before_date +'_3.0'])
    user_actionClass[before_date1 + '购买/收藏转化率'] =  np.log(1 + user_actionClass[before_date + '_4.0']) - np.log(1 + user_actionClass[before_date +'_5.0'])
    user_actionClass[before_date1 + '购买/点击转化率'] =  np.log(1 + user_actionClass[before_date + '_4.0']) - np.log(1 + user_actionClass[before_date +'_6.0'])
     
    #计算均值，统计用户每天不同行为数量
    user_actionClass[before_date1 +'浏览行为均值'] = user_actionClass[before_date + '_1.0'] / day
    user_actionClass[before_date1 +'加购行为均值'] = user_actionClass[before_date + '_2.0'] / day
    user_actionClass[before_date1 +'删购行为均值'] = user_actionClass[before_date + '_3.0'] / day
    user_actionClass[before_date1 +'购买行为均值'] = user_actionClass[before_date + '_4.0'] / day
    user_actionClass[before_date1 +'收藏行为均值'] = user_actionClass[before_date + '_5.0'] / day
    user_actionClass[before_date1 +'点击行为均值'] = user_actionClass[before_date + '_6.0'] / day

    return user_actionClass

3.4.2.最终处理结果如下：首先展示最终的到的用户ID-行为总数特征表

In [ ]:
day = 3
get_accumulate_user_feature('2016-02-04', all_actions, day)#提取用户ID-行为总数特征

3.4.3.分步实现get_accumulate_user_feature函数的过程如下：

3.4.3.1.首先展示原始数据

In [ ]:
all_actions#2、3、4月用户行为数据合集

3.4.3.2.设置一个长达3天的时间片段的起始时间和终止时间

In [ ]:
train_start_date = '2016-02-01'
train_end_date = datetime.strptime(train_start_date, '%Y-%m-%d') + timedelta(days=3)
train_end_date = train_end_date.strftime('%Y-%m-%d')
print (train_start_date)
print (train_end_date)

3.4.3.3.读取上述时间片段内的数据

In [ ]:
actions = get_actions(train_start_date, train_end_date, all_actions)#调用get_actions函数，读取数据
actions

3.4.3.4.对“行为类别”列做一个独热编码

In [ ]:
before_date = '%s天行为总数' % day#给定列名字符串
before_date#定义列名的字符串
dummies_actionClass = pd.get_dummies(actions['行为类别'], prefix=before_date)#对“行为类别”列做一个独热编码
dummies_actionClass

3.4.3.5.统计3天内不同行为总数

In [ ]:
actions['日期'] = pd.to_datetime(actions['时间']).apply(lambda x: x.date())#转换时间格式
user_actionClass = pd.concat([actions[['用户ID', '日期']], dummies_actionClass], axis=1)
user_actionClass = user_actionClass.groupby(['用户ID'], as_index=False).sum()#统计用户3天内不同行为总数
user_actionClass

3.4.3.6.计算用户3天内购买-浏览行为转化率

In [ ]:
#使用np.log做减法，相当于做除法，得到的结果就是购买、浏览的比重
before_date1 = '%s天' %3
user_actionClass[before_date1 + '购买/浏览行为转化率'] =  np.log(1 + user_actionClass[before_date + '_4.0']) - np.log(1 + user_actionClass[before_date +'_1.0'])
user_actionClass

3.4.3.7.计算用户3太难内不同行为的均值，以浏览为例

In [ ]:
user_actionClass[before_date1 +'浏览行为均值'] = user_actionClass[before_date + '_1.0'] / day
user_actionClass

**3.5.倒数3天、30天，用户ID-行为总数的特征提取**

在上面用户Id-行为总数特征表的基础上，分别提取用户近一个月、近三天的特征，然后计算一个月内用户出去最近三天的行为占据一个月的行为的比重。

3.5.1.原始数据如下：

In [ ]:
all_actions#2、3、4月用户行为数据合集

3.5.2.实现函数如下：

In [ ]:
#定义get_recent_user_feature函数，提取倒数3天，30天，用户ID-行为总数特征
def get_recent_user_feature(end_date, all_actions):
    user_actionClass_3days = get_accumulate_user_feature(end_date, all_actions, 3)#调用函数，构建用户近3天的用户ID-行为总数特征表
    user_actionClass_30days = get_accumulate_user_feature(end_date, all_actions, 30)#调用函数，构建用户近30天的用户ID-行为总数特征表
    user_actionClass = pd.merge(user_actionClass_3days, user_actionClass_30days, how ='left', on='用户ID')#将上述得到的两个特征表连接
    
    #计算用户27天各行为总数（用户近30天各行为总数减去近3天的各行为总数）在近30天各行为总数中的占比
    before_date3=before_date1 = '%s天行为总数' %3
    before_date30=before_date1 = '%s天行为总数' %30
    user_actionClass['27天浏览行为占比']=np.log(1 + user_actionClass[before_date30+'_1.0']-user_actionClass[before_date3+'_1.0'])-np.log(1 + user_actionClass[before_date30+'_1.0'])
    user_actionClass['27天加购行为占比']=np.log(1 + user_actionClass[before_date30+'_2.0']-user_actionClass[before_date3+'_2.0'])-np.log(1 + user_actionClass[before_date30+'_2.0'])
    user_actionClass['27天删购行为占比']=np.log(1 + user_actionClass[before_date30+'_3.0']-user_actionClass[before_date3+'_3.0'])-np.log(1 + user_actionClass[before_date30+'_3.0'])
    user_actionClass['27天购买行为占比']=np.log(1 + user_actionClass[before_date30+'_4.0']-user_actionClass[before_date3+'_4.0'])-np.log(1 + user_actionClass[before_date30+'_4.0'])
    user_actionClass['27天收藏行为占比']=np.log(1 + user_actionClass[before_date30+'_5.0']-user_actionClass[before_date3+'_5.0'])-np.log(1 + user_actionClass[before_date30+'_5.0'])
    user_actionClass['27天点击行为占比']=np.log(1 + user_actionClass[before_date30+'_6.0']-user_actionClass[before_date3+'_6.0'])-np.log(1 + user_actionClass[before_date30+'_6.0'])
    return user_actionClass

3.5.3.最终处理结果如下：展示倒数3太难、30天，用户ID-行为总数特征表

In [ ]:
train_end_date='2016-03-01'
get_recent_user_feature(train_end_date, all_actions) #调用get_recent_user_feature函数，提取倒数3天、30天，用户ID-行为总数特征

**3.6.用户ID-商品类别-行为类别-行为总数的特征提取**

统计用户对各个类别的各项行为操作数量，统计用户对各个类别操作行为数量占比对所有类别操作行为数量的比重。

In [ ]:
#定义get_user_productClass_feature函数，提取用户ID-商品类别-行为总数特征
def get_user_productClass_feature(start_date, end_date, all_actions):
    
    actions = get_actions(start_date, end_date, all_actions)#获取起始时间-终止时间片段内的所有数据
    actions_3columns = actions[['用户ID', '商品类别', '行为类别']]#保留“用户ID”、“商品类别”、“行为类别”这3列
    dummies_actionClass= pd.get_dummies(actions['行为类别'], prefix='行为')#对“行为类别”作独热编码
    productClass_actionClass = pd.concat([actions[['用户ID', '商品类别']], dummies_actionClass], axis=1)#将各特征按列连接
    productClass_actionClass = productClass_actionClass.groupby(['用户ID', '商品类别']).sum()#统计用户对各个类别商品的各项行为总数
    
    #将两个索引信息放在一个新索引中
    productClass_actionClass = productClass_actionClass.unstack()#使用unstack函数，将行索引变成列索引
    productClass_actionClass.columns = productClass_actionClass.columns.swaplevel(0, 1)#调换索引列的顺序
    productClass_actionClass.columns = productClass_actionClass.columns.droplevel()#删除“商品类别”索引列
    #c4、c6、……、c11：依次代表第4类、第5类、……、第11类商品
    #a1、a2、……a6：依次代表浏览、加购物车、删购物车、购买、收藏、点击这6个行为
    #c4a1sum：代表第4类商品浏览行为总数
    productClass_actionClass.columns = [
        'c4a1sum', 'c5a1sum', 'c6a1sum', 'c7a1sum', 'c8a1sum', 'c9a1sum', 'c10a1sum', 'c11a1sum',
        'c4a2sum', 'c5a2sum', 'c6a2sum', 'c7a2sum', 'c8a2sum', 'c9a2sum', 'c10a2sum', 'c11a2sum',
        'c4a3sum', 'c5a3sum', 'c6a3sum', 'c7a3sum', 'c8a3sum', 'c9a3sum', 'c10a3sum', 'c11a3sum',
        'c4a4sum', 'c5a4sum', 'c6a4sum', 'c7a4sum', 'c8a4sum', 'c9a4sum', 'c10a4sum', 'c11a4sum',
        'c4a5sum', 'c5a5sum', 'c6a5sum', 'c7a5sum', 'c8a5sum', 'c9a5sum', 'c10a5sum', 'c11a5sum',
        'c4a6sum', 'c5a6sum', 'c6a6sum', 'c7a6sum', 'c8a6sum', 'c9a6sum', 'c10a6sum', 'c11a6sum',
    ]
    
    productClass_actionClass = productClass_actionClass.fillna(0)#将空值填充为0
    productClass_actionClass['sum'] = productClass_actionClass.sum(axis=1)#统计sum（用户各行为总数）
    
    #计算用户对各商品类别行为总数在用户各行为总数中的占比
    #用户对第4类商品的各类行为总数占用用户各行为总数的比重
    productClass_actionClass['c4sum/sum'] = (
    productClass_actionClass['c4a1sum']+productClass_actionClass['c4a2sum']+productClass_actionClass['c4a3sum']+productClass_actionClass['c4a4sum']+productClass_actionClass['c4a5sum']+ productClass_actionClass['c4a6sum'] )/productClass_actionClass['sum']
    #用户对第5类商品的各类行为总数占用用户各行为总数的比重
    productClass_actionClass['c5sum/sum'] = (
    productClass_actionClass['c5a1sum']+productClass_actionClass['c5a2sum']+productClass_actionClass['c5a3sum']+productClass_actionClass['c5a4sum']+productClass_actionClass['c5a5sum']+ productClass_actionClass['c5a6sum'] )/productClass_actionClass['sum']
    #用户对第6类商品的各类行为总数占用用户各行为总数的比重
    productClass_actionClass['c6sum/sum'] = (
    productClass_actionClass['c6a1sum']+productClass_actionClass['c6a2sum']+productClass_actionClass['c6a3sum']+productClass_actionClass['c6a4sum']+productClass_actionClass['c6a5sum']+ productClass_actionClass['c6a6sum'] )/productClass_actionClass['sum']
    #用户对第7类商品的各类行为总数占用用户各行为总数的比重
    productClass_actionClass['c7sum/sum'] = (
    productClass_actionClass['c7a1sum']+productClass_actionClass['c7a2sum']+productClass_actionClass['c7a3sum']+productClass_actionClass['c7a4sum']+productClass_actionClass['c7a5sum']+ productClass_actionClass['c7a6sum'] )/productClass_actionClass['sum']
    #用户对第8类商品的各类行为总数占用用户各行为总数的比重
    productClass_actionClass['c8sum/sum'] = (
    productClass_actionClass['c8a1sum']+productClass_actionClass['c8a2sum']+productClass_actionClass['c8a3sum']+productClass_actionClass['c8a4sum']+productClass_actionClass['c8a5sum']+ productClass_actionClass['c8a6sum'] )/productClass_actionClass['sum']
    #用户对第9类商品的各类行为总数占用用户各行为总数的比重
    productClass_actionClass['c9sum/sum'] = (
    productClass_actionClass['c9a1sum']+productClass_actionClass['c9a2sum']+productClass_actionClass['c9a3sum']+productClass_actionClass['c9a4sum']+productClass_actionClass['c9a5sum']+ productClass_actionClass['c9a6sum'] )/productClass_actionClass['sum']
    #用户对第10类商品的各类行为总数占用用户各行为总数的比重
    productClass_actionClass['c10sum/sum'] = (
    productClass_actionClass['c10a1sum']+productClass_actionClass['c10a2sum']+productClass_actionClass['c10a3sum']+productClass_actionClass['c10a4sum']+productClass_actionClass['c10a5sum']+ productClass_actionClass['c10a6sum'] )/productClass_actionClass['sum']
    #用户对第11类商品的各类行为总数占用用户各行为总数的比重
    productClass_actionClass['c11sum/sum'] = (
    productClass_actionClass['c11a1sum']+productClass_actionClass['c11a2sum']+productClass_actionClass['c11a3sum']+productClass_actionClass['c11a4sum']+productClass_actionClass['c11a5sum']+ productClass_actionClass['c11a6sum'] )/productClass_actionClass['sum']
  
    #计算用户对第8类商品的不同行为总数在用户不同行为总数中的占比
    #c8a1sum/a1sum：代表第8类商品浏览行为在用户所有浏览行为中占比
    productClass_actionClass['c8a1sunm/a1sum'] = np.log(
        1 + productClass_actionClass['c8a1sum']) - np.log(
            1 + productClass_actionClass['c8a1sum'] + productClass_actionClass['c4a1sum'] +productClass_actionClass['c5a1sum'] + productClass_actionClass['c6a1sum'] +
            productClass_actionClass['c7a1sum'] + productClass_actionClass['c9a1sum'] + productClass_actionClass['c10a1sum'] + productClass_actionClass['c11a1sum'])
    #c8a2sum/a2sum：代表第8类商品加购物车行为在用户所有加购物车行为中占比
    productClass_actionClass['c8a2sunm/a2sum'] = np.log(
        1 + productClass_actionClass['c8a2sum']) - np.log(
            1 + productClass_actionClass['c8a2sum'] + productClass_actionClass['c4a2sum'] +productClass_actionClass['c5a2sum'] + productClass_actionClass['c6a2sum'] +
           productClass_actionClass['c7a2sum'] + productClass_actionClass['c9a2sum'] +productClass_actionClass['c10a2sum'] + productClass_actionClass['c11a2sum'])
    #c8a3sum/a3sum：代表第8类商品删购物车行为在用户所有删购物车行为中占比
    productClass_actionClass['c8a3sunm/a3sum'] = np.log(
        1 + productClass_actionClass['c8a3sum']) - np.log(
            1 + productClass_actionClass['c8a3sum'] + productClass_actionClass['c4a3sum'] +productClass_actionClass['c5a3sum'] + productClass_actionClass['c6a3sum'] +
            productClass_actionClass['c7a3sum'] + productClass_actionClass['c9a3sum'] + productClass_actionClass['c10a3sum'] + productClass_actionClass['c11a3sum'])
    #c8a4sum/a4sum：代表第8类商品购买行为在用户所有购买行为中占比
    productClass_actionClass['c8a4sunm/a4sum'] = np.log(
        1 + productClass_actionClass['c8a4sum']) - np.log(
            1 + productClass_actionClass['c8a4sum'] + productClass_actionClass['c4a4sum'] +productClass_actionClass['c5a4sum'] + productClass_actionClass['c6a4sum'] +
            productClass_actionClass['c7a4sum'] + productClass_actionClass['c9a4sum'] + productClass_actionClass['c10a4sum'] + productClass_actionClass['c11a4sum'])
    #c8a5sum/a5sum：代表第8类商品收藏行为在用户所有收藏行为中占比
    productClass_actionClass['c8a5sunm/a5sum'] = np.log(
        1 + productClass_actionClass['c8a5sum']) - np.log(
            1 + productClass_actionClass['c8a5sum'] + productClass_actionClass['c4a5sum'] +productClass_actionClass['c5a5sum'] + productClass_actionClass['c6a5sum'] +
            productClass_actionClass['c7a5sum'] + productClass_actionClass['c9a5sum'] + productClass_actionClass['c10a5sum'] + productClass_actionClass['c11a5sum'])
    #c8a6sum/a6sum：代表第8类商品点击行为在用户所有点击行为中占比
    productClass_actionClass['c8a6sunm/a6sum'] = np.log(
        1 + productClass_actionClass['c8a6sum']) - np.log(
            1 + productClass_actionClass['c8a6sum'] + productClass_actionClass['c4a6sum'] +productClass_actionClass['c5a6sum'] + productClass_actionClass['c6a6sum'] +
            productClass_actionClass['c7a6sum'] + productClass_actionClass['c9a6sum'] + productClass_actionClass['c10a6sum'] + productClass_actionClass['c11a6sum'])

    productClass_actionClass['用户ID'] = productClass_actionClass.index
    productClass_actionClass = productClass_actionClass[[
        '用户ID', 
        'c4sum/sum', 'c5sum/sum', 'c6sum/sum', 'c7sum/sum', 'c8sum/sum', 'c9sum/sum', 'c10sum/sum', 'c11sum/sum', 
        'c8a1sunm/a1sum','c8a2sunm/a2sum', 'c8a3sunm/a3sum', 'c8a4sunm/a4sum', 'c8a5sunm/a5sum', 'c8a6sunm/a6sum']]
    productClass_actionClass = productClass_actionClass.drop(['用户ID'], axis = 1).reset_index()#重置索引

    return productClass_actionClass

3.6.2.最终处理结果如下：首先展示最终用户ID-商品类别-行为类别-行为总数特征表：

In [ ]:
#在此仅读取长达3太难的时间片段内的数据，以举例展示最终得到的用户Id-商品类别-行为类别-行为总数特征表
train_start_date = '2016-02-01'
train_end_date = datetime.strptime(train_start_date, '%Y-%m-%d') + timedelta(days=3)
train_end_date = train_end_date.strftime('%Y-%m-%d')
get_user_productClass_feature(train_start_date, train_end_date, all_actions) #调用get_user_productClass_feature函数，提取用户ID-商品类别-行为类别-行为总数特征

3.6.3.分步实现get_user_productClass_feature()函数的过程如下：

3.6.3.1.首先展示原始数据

In [ ]:
all_actions

3.6.3.2.设置一个长达3天的时间片段的起始时间和终止时间

In [ ]:
train_start_date = '2016-02-01'
train_end_date = datetime.strptime(train_start_date, '%Y-%m-%d') + timedelta(days=3)
train_end_date = train_end_date.strftime('%Y-%m-%d')
print (train_start_date)
print (train_end_date)

3.6.3.3.读取上述时间片段内的数据

In [ ]:
actions = get_actions(train_start_date, train_end_date, all_actions)#调用get_actions函数读取数据
actions_3columns = actions[['用户ID', '商品类别', '行为类别']]#保存'用户ID', '商品类别', '行为类别'三列数据
actions_3columns

3.6.3.4.统计不同用户对不同种类执行不同行为的次数

In [ ]:
dummies_actionClass = pd.get_dummies(actions['行为类别'], prefix='行为')#对“行为类别”做一个独热编码
productClass_actionClass = pd.concat([actions[['用户ID', '商品类别']], dummies_actionClass], axis=1)#连接各特征列
productClass_actionClass = productClass_actionClass.groupby(['用户ID', '商品类别']).sum()#统计不同用户对不同种类商品执行不同行为的次数
productClass_actionClass

3.6.3.5.将两个索引信息，处理成一个新索引

(1).使用unstack函数，将行索引变成列索引

In [ ]:
productClass_actionClass = productClass_actionClass.unstack()#使用unstack函数，将行索引变成列索引
productClass_actionClass

(2).查看索引列名称

In [ ]:
productClass_actionClass.columns#查看索引列名称

(3).调换索引列的顺序

In [ ]:
productClass_actionClass.columns = productClass_actionClass.columns.swaplevel(0, 1)#调换索引列的顺序
productClass_actionClass.columns

(4).展示调整索引列顺序后的数据集

In [ ]:
productClass_actionClass

(6).删除“商品类别”索引列

In [ ]:
productClass_actionClass.columns = productClass_actionClass.columns.droplevel()#删除“商品类别”索引列
productClass_actionClass.columns

(7).将两个原索引信息，放在一个新索引里

In [ ]:
#c4、c6、……、c11：依次代表第4类、第5类、……、第11类商品
#a1、a2、……a6：依次代表浏览、加购物车、删购物车、购买、收藏、点击这6个行为
#c4a1sum：代表第4类商品浏览行为总数
productClass_actionClass.columns = [
        'c4a1sum', 'c5a1sum', 'c6a1sum', 'c7a1sum', 'c8a1sum', 'c9a1sum', 'c10a1sum', 'c11a1sum',
        'c4a2sum', 'c5a2sum', 'c6a2sum', 'c7a2sum', 'c8a2sum', 'c9a2sum', 'c10a2sum', 'c11a2sum',
        'c4a3sum', 'c5a3sum', 'c6a3sum', 'c7a3sum', 'c8a3sum', 'c9a3sum', 'c10a3sum', 'c11a3sum',
        'c4a4sum', 'c5a4sum', 'c6a4sum', 'c7a4sum', 'c8a4sum', 'c9a4sum', 'c10a4sum', 'c11a4sum',
        'c4a5sum', 'c5a5sum', 'c6a5sum', 'c7a5sum', 'c8a5sum', 'c9a5sum', 'c10a5sum', 'c11a5sum',
        'c4a6sum', 'c5a6sum', 'c6a6sum', 'c7a6sum', 'c8a6sum', 'c9a6sum', 'c10a6sum', 'c11a6sum',
    ]
productClass_actionClass.columns

(8).展示最终的使用新索引的数据集

In [ ]:
productClass_actionClass

3.6.3.6.填充缺失值，添加新特征“用户各行为数据”

In [ ]:
productClass_actionClass = productClass_actionClass.fillna(0)#填充缺失值，填充为0
productClass_actionClass['sum'] = productClass_actionClass.sum(axis=1)#计算sum（用户各种行为总数）
productClass_actionClass

3.6.3.7.计算用户对第8类商品的各种行为占用户行为总数的比重

In [ ]:
#c8sum/sum：代表用户对第8类商品所有行为总数占用户行为总数的比重
productClass_actionClass['c8sum/sum'] = (
    productClass_actionClass['c8a1sum']+productClass_actionClass['c8a2sum']+productClass_actionClass['c8a3sum']+productClass_actionClass['c8a4sum']+productClass_actionClass['c8a5sum']+productClass_actionClass['c8a6sum']) / productClass_actionClass['sum']
productClass_actionClass

3.6.3.8.计算用户对第8类商品的浏览行为总数在用户浏览行为总数中所占比重

In [ ]:
productClass_actionClass['c8a1sum/a1sum'] = np.log(
        1 + productClass_actionClass['c8a1sum']) - np.log(
            1 + productClass_actionClass['c8a1sum'] + productClass_actionClass['c4a1sum'] +productClass_actionClass['c5a1sum'] + productClass_actionClass['c6a1sum'] +
            productClass_actionClass['c7a1sum'] + productClass_actionClass['c9a1sum'] + productClass_actionClass['c10a1sum'] + productClass_actionClass['c11a1sum'])
productClass_actionClass    

**3.7.商品ID-行为类别-行为总数的特征提取**

分时间段，统计用户对商品的不同行为的数量、购买转化率、均值

3.7.1.实现函数如下：

In [ ]:
#定义一个get_accumulate_product_feature函数，提取商品ID-行为类别-行为总数特征
def get_accumulate_product_feature(start_date, end_date, all_actions):
    
    actions = get_actions(start_date, end_date, all_actions)#读取起始时间-终止时间片段内的所有数据
    dummies_actionClass = pd.get_dummies(actions['行为类别'], prefix='商品-行为总数')#对“行为类别”作独热编码，并指定前缀
    actions['日期'] = pd.to_datetime(actions['时间']).apply(lambda x: x.date())#转换时间格式
    product_actionClass = pd.concat([actions[['商品ID', '日期']], dummies_actionClass], axis=1)#将各特征按列连接
    product_actionClass = product_actionClass.groupby(['商品ID'], as_index=False).sum()#统计不同商品被用户执行各类操作的次数
    
    #计算商品的购买/浏览转化率、购买/加购转化率、购买/删除转化率、购买/收藏转化率、购买/点击转化率
    before_date='商品-行为总数'#定义列名字符串
    product_actionClass['购买/浏览转化率'] =  np.log(1 + product_actionClass[before_date+'_4.0']) - np.log(1 + product_actionClass[before_date+'_1.0'])
    product_actionClass['购买/加购转化率'] =  np.log(1 + product_actionClass[before_date+'_4.0']) - np.log(1 + product_actionClass[before_date+'_2.0'])
    product_actionClass['购买/删购转化率'] =  np.log(1 + product_actionClass[before_date+'_4.0']) - np.log(1 + product_actionClass[before_date+'_3.0'])
    product_actionClass['购买/收藏转化率'] =  np.log(1 + product_actionClass[before_date+'_4.0']) - np.log(1 + product_actionClass[before_date+'_5.0'])
    product_actionClass['购买/点击转化率'] =  np.log(1 + product_actionClass[before_date+'_4.0']) - np.log(1 + product_actionClass[before_date+'_6.0'])
    
    #计算各种行为的均值
    days_interal=(datetime.strptime(end_date, '%Y-%m-%d')-datetime.strptime(start_date, '%Y-%m-%d')).days#计算终止时间-起止时间之间相差的天数
    product_actionClass['商品-浏览行为均值'] = product_actionClass[
        before_date+'_1.0'] / days_interal
    product_actionClass['商品-加购行为均值'] = product_actionClass[
        before_date+'_2.0'] / days_interal
    product_actionClass['商品-删购行为均值'] = product_actionClass[
        before_date+'_3.0'] / days_interal
    product_actionClass['商品-购买行为均值'] = product_actionClass[
        before_date+'_4.0'] / days_interal
    product_actionClass['商品-收藏行为均值'] = product_actionClass[
        before_date+'_5.0'] / days_interal
    product_actionClass['商品-点击行为均值'] = product_actionClass[
        before_date+'_6.0'] / days_interal
    return product_actionClass

3.7.2.最终处理结果如下：首先展示最终的商品ID-行为类别-行为总数特征表：

In [ ]:
#调用get_accumulate_product_feature函数，提取商品ID-行为类别-行为总数特征
train_start_date = '2016-02-01'
train_end_date = datetime.strptime(train_start_date, '%Y-%m-%d') + timedelta(days=3)
train_end_date = train_end_date.strftime('%Y-%m-%d')
get_accumulate_product_feature(train_start_date, train_end_date, all_actions)

3.7.3.分布实现get_accumulate_product_feature()函数的过程如下：

3.7.3.1.首先展示原始数据

In [ ]:
all_actions

3.7.3.2.设置一个长达3天的时间片段的起始时间和终止时间

In [ ]:
train_start_date = '2016-02-01'
train_end_date = datetime.strptime(train_start_date, '%Y-%m-%d') + timedelta(days=3)
train_end_date = train_end_date.strftime('%Y-%m-%d')
print (start_date)
print (train_end_date)

3.7.3.3.读取上述时间片段内的数据并处理

In [ ]:
actions = get_actions(train_start_date, train_end_date, all_actions)#读取起始时间-终止时间片段内的所有数据
dummies_actionClass = pd.get_dummies(actions['行为类别'], prefix='商品-行为总数')#对“行为类别”做独热编码，指定前缀
actions['日期'] = pd.to_datetime(actions['时间']).apply(lambda x: x.date())#转换时间格式
product_actionClass = pd.concat([actions[['商品ID', '日期']], dummies_actionClass], axis=1)#连接各特征列
product_actionClass

3.7.3.4.统计不同商品被用户执行各类操作的次数

In [ ]:
product_actionClass = product_actionClass.groupby(['商品ID'], as_index=False).sum()
product_actionClass

3.7.3.5.计算购买-各类行为转化率，以购买-浏览转化率为例

In [ ]:
before_date='商品-行为总数'
product_actionClass['购买/浏览转化率'] =  np.log(1 + product_actionClass[before_date+'_4.0']) - np.log(1 + product_actionClass[before_date+'_1.0'])
product_actionClass

**3.8.商品类别-行为总数、转化率、日均行为总数的特征提取**

计算不同时间段下，各个商品类别的购买转化率、均值

3.8.1.原始数据如下

In [ ]:
all_actions

3.8.2.实现函数如下：

In [ ]:
#定义get_accumulate_productClass_feature函数，提取商品类别-行为总数、转化率、日均行为总数特征
def get_accumulate_productClass_feature(start_date, end_date, all_actions):
   
    actions = get_actions(start_date, end_date, all_actions)#读取起始时间-终止时间片段内的所有数据
    actions['日期'] = pd.to_datetime(actions['时间']).apply(lambda x: x.date())#转换时间格式
    dummies_actionClass = pd.get_dummies(actions['行为类别'], prefix='商品类别-行为总数')#对“行为类别”做独热编码，设置前缀
    productClass_actionClass = pd.concat([actions[['商品类别','日期']], dummies_actionClass], axis=1)#将各特征按列连接
    productClass_actionClass = productClass_actionClass.groupby(['商品类别'], as_index=False).sum()#统计商品类别-行为总数
    
    #计算商品类别-行为转化率
    productClass_actionClass['购买/浏览转化率'] =(np.log(1 + productClass_actionClass['商品类别-行为总数_4.0']) - np.log(1 + productClass_actionClass['商品类别-行为总数_1.0']))
    productClass_actionClass['购买/加购转化率'] =(np.log(1 + productClass_actionClass['商品类别-行为总数_4.0']) - np.log(1 + productClass_actionClass['商品类别-行为总数_2.0']))
    productClass_actionClass['购买/删购转化率'] =(np.log(1 + productClass_actionClass['商品类别-行为总数_4.0']) - np.log(1 + productClass_actionClass['商品类别-行为总数_3.0']))
    productClass_actionClass['购买/收藏转化率'] =(np.log(1 + productClass_actionClass['商品类别-行为总数_4.0']) - np.log(1 + productClass_actionClass['商品类别-行为总数_5.0']))
    productClass_actionClass['购买/点击转化率'] =(np.log(1 + productClass_actionClass['商品类别-行为总数_4.0']) - np.log(1 + productClass_actionClass['商品类别-行为总数_6.0']))
    
    #计算给定时间段内用户对各个类别商品执行不同操作次数的均值
    days_interal=(datetime.strptime(end_date, '%Y-%m-%d')-datetime.strptime(start_date, '%Y-%m-%d')).days #计算终止时间-起始时间之间相差的天数
    productClass_actionClass['商品类别-浏览行为均值'] = productClass_actionClass['商品类别-行为总数_1.0'] /  days_interal
    productClass_actionClass['商品类别-加购行为均值'] = productClass_actionClass['商品类别-行为总数_2.0'] /  days_interal
    productClass_actionClass['商品类别-删购行为均值'] = productClass_actionClass['商品类别-行为总数_3.0'] /  days_interal
    productClass_actionClass['商品类别-购买行为均值'] = productClass_actionClass['商品类别-行为总数_4.0'] /  days_interal
    productClass_actionClass['商品类别-收藏行为均值'] = productClass_actionClass['商品类别-行为总数_5.0'] /  days_interal
    productClass_actionClass['商品类别-点击行为均值'] = productClass_actionClass['商品类别-行为总数_6.0'] /  days_interal
    return productClass_actionClass

3.8.3.最终处理结果如下：首先展示最终的商品类别-行为总数、转化率、日均行为总数特征表

In [ ]:
#调用get_accumulate_productClass_feature函数，提取商品类别-行为总数、转化率、日均行为总数特征
train_start_date = '2016-02-01'
train_end_date = datetime.strptime(train_start_date, '%Y-%m-%d') + timedelta(days=3)
train_end_date = train_end_date.strftime('%Y-%m-%d')
get_accumulate_productClass_feature(train_start_date, train_end_date, all_actions)

**3.9.构造训练集**

3.9.1提取标签

3.9.1.1.原始数据如下：

In [ ]:
 all_actions

3.9.1.2.实现函数如下：

In [ ]:
#调用get_labels函数，购买第8类商品的用户，并打上标签1
def get_labels(start_date, end_date, all_actions):
    actions = get_actions(start_date, end_date, all_actions)#调用get_actions函数，读取起始时间-终止时间片段内的数据
    actions = actions[(actions['行为类别'] == 4) & (actions['商品类别']==8)]#获取购买第8类商品的用户，行为类别=4为购买
    actions = actions.groupby(['用户ID', '商品ID'], as_index=False).sum()#统计购买第8类商品的用户-商品组合数据
    actions['标签'] = 1 #将购买第8类商品的用户打上标签1
    actions = actions[['用户ID', '商品ID', '标签']]#保留“用户ID”、“商品ID”、“标签”这三列数据
    return actions

3.9.1.3.最终处理结果如下：首先展示最终得到的标签数据

In [ ]:
#设置一个长达3天的时间片段的起始时间和终止时间
train_start_date = '2016-02-01'
train_end_date = datetime.strptime(train_start_date, '%Y-%m-%d') + timedelta(days=3)
train_end_date = train_end_date.strftime('%Y-%m-%d')
get_labels(train_start_date, train_end_date, all_actions)#以长达3天的时间片段内的数据举例，展示最终得到的标签数据

3.9.2.合并所有特征表，确定样本量

In [ ]:
#定义 make_actions函数，合并所有特征，确定样本量
def make_actions(user, product, all_actions, start_date):
    
    #timedalte是datetime中的一个对象，该对象表示两个时间的差值
    end_date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=3)#设置长达3天的时间片段的终止时间
    end_date = end_date.strftime('%Y-%m-%d')#转换时间格式
    print (end_date)
    start_date_30day = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=30)#设置长达30天的时间片段的起始时间
    start_date_30day = start_date_30day.strftime('%Y-%m-%d')#转换时间格式
    
    #调用get_recent_user_feature函数，调用倒数3天、30天，用户ID-行为总数特征
    user_action_feature = get_recent_user_feature(end_date, all_actions)
    print ('get_recent_user_feature finished')
    #调用get_user_productClass_feature函数，提取用户ID-商品类别-行为类别-行为总数特征
    user_productClass_feature = get_user_productClass_feature(start_date, end_date, all_actions)
    print ('get_user_productClass_feature finished')
    #get_accumulate_product_feature函数，提取商品ID-行为类别-行为总数特征
    accumulate_product_feature = get_accumulate_product_feature(start_date_30day, end_date, all_actions)
    print ('get_accumulate_product_feature finished')
    #调用get_accumulate_productClass_feature函数，提取商品类别-行为总数、转化率、日均行为总数特征
    accumulate_productClass_feature = get_accumulate_productClass_feature(start_date_30day, end_date, all_actions)
    print ('get_accumulate_productClass_feature finished')
    #调用get_comments_feature函数，提取评论特征
    comment_feature = get_comments_feature(end_date)
    print ('get_comments_product_feat finished')
    
    #设置一个长达5天的时间片段的起始时间和终止时间
    test_start_date = end_date
    test_end_date = datetime.strptime(test_start_date, '%Y-%m-%d') + timedelta(days=5)
    test_end_date = test_end_date.strftime('%Y-%m-%d')
    
    #调用get_labels函数，提取购买第8类商品的“用户ID”、“商品ID”、“标签”这三列特征
    labels = get_labels(test_start_date, test_end_date, all_actions)
    print ("get labels")
    
    actions = None
    #设置7个长达3、5、……、30天的时间片段的起始时间和终止时间
    for i in (3, 5, 7, 10, 15, 21, 30):
        start_date_30day = datetime.strptime(train_end_date, '%Y-%m-%d') - timedelta(days=i)
        start_date_30day = start_date_30day.strftime('%Y-%m-%d')
        if actions is None:
            #调用get_action_feature函数，提取用户ID-商品ID-行为总数特征 
            actions = get_action_feature(start_date_30day, end_date, all_actions, i)
        else:
            #连接各特征表
            actions = pd.merge(actions, get_action_feature(start_date_30day, end_date, all_actions, i), how='left',
                               on=['用户ID', '商品ID', '商品类别']) 
    #依次连接上述的到的各特征表
    actions = pd.merge(actions, user, how='left', on='用户ID')
    actions = pd.merge(actions, user_action_feature, how='left', on='用户ID')
    actions = pd.merge(actions, user_productClass_feature, how='left', on='用户ID')
    actions = pd.merge(actions, product, how='left', on=['商品ID', '商品类别'])
    actions = pd.merge(actions, accumulate_product_feature, how='left', on='商品ID')
    actions = pd.merge(actions, accumulate_productClass_feature, how='left', on='商品类别')
    actions = pd.merge(actions, comment_feature, how='left', on='商品ID')
    actions = pd.merge(actions, labels, how='left', on=['用户ID', '商品ID'])
    
    actions = actions.fillna(0)#填充连接后的特征表（actions）的空值
    action_postive = actions[actions['标签'] == 1]#提取标签数据集
    action_negative = actions[actions['标签'] == 0]#提取特征数据集
    del actions#删除变量
    
    #将购买第8类商品的用户数量的10倍，作为抽取特征数据集的样本数量
    neg_len = len(action_postive) * 10 
    action_negative = action_negative.sample(n=neg_len)
    action_sample = pd.concat([action_postive, action_negative], ignore_index=True)#将两个表连接    
    return action_sample

3.9.3.定义构造训练集的函数

In [ ]:
#定义make_train_set函数，构造训练集
def make_train_set(start_date, setNums ,f_path, all_actions):
    train_actions = None
    
    #调用get_user_feature函数，提取用户基本特征
    user = get_user_feature(user_base_info)
    print ('get_user_feature finished')
    #调用get_product_feature函数，提取商品基本特征
    product = pd.read_csv(product_path, engine='python', encoding='utf-8')
    product = get_product_feature(product)
    print ('get_product_feature finished')
    
    #构造多组训练数据，设置setNums个提取特征的时间片段的起始时间
    for i in range(setNums):
        print (start_date)
        if train_actions is None:
            #调用make_actions函数合并所有特征
            train_actions = make_actions(user, product, all_actions, start_date)
        else:
            train_actions = pd.concat([train_actions, make_actions(user, product, all_actions, start_date)],
                                          ignore_index=True)
        #设置时间片段的起始时间每次移动一天
        start_date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
        print ("round {0}/{1} over!".format(i+1, setNums))#打印当前结束的循环次数
        
    train_actions.to_csv(f_path, index=False,encoding = 'utf-8')#将训练集保存

In [ ]:
start_date = '2016-02-01'#设置起始时间
make_train_set(start_date,20, 'data/result_训练集.csv',all_actions)#执行make_train_set函数，构造训练集

展示最终得到的训练集

In [ ]:
training_set = pd.read_csv('data/result_训练集.csv',engine='python',encoding = 'utf-8')#读取最终得到的训练集
training_set#展示读取的数据

**3.10.构造验证集**

3.10.1.编写构造验证集的函数make_validation_set()

In [ ]:
#定义 make_validation_data函数，获取给定时间片段内购买第8类商品的数据集
def make_validation_data(validation_start_date, validation_end_date, all_actions, label_validation_path):
    actions = get_actions(validation_start_date, validation_end_date,all_actions)#调用get_actions函数，读取给定时间片段内的数据
    actions = actions[(actions['行为类别'] == 4) & (actions['商品类别'] == 8)]#提取购买第8类商品的用户数据
    actions = actions[['用户ID', '商品ID']]#提取“用户ID”、“商品ID”两列数据
    actions = actions.drop_duplicates()#丢弃重复行
    actions.to_csv('data/result_'.format(label_validation_path), index=False)

#定义make_validation_set函数，合并所有特征，构造验证集
def make_validation_set(start_date, end_date, validation_path,all_actions ):  
    #设置长达30天的时间片段的起始时间
    start_date_30day = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=30)
    start_date_30day = start_date_30day.strftime('%Y-%m-%d')
    print ("get all actions!")    
    #调用get_user_feature函数，提取用户基本特征
    user = get_user_feature(user_base_info)
    print ('get_user_feature finished')
    #调用get_product_feature，提取商品基本特征
    product = pd.read_csv(product_path, engine='python', encoding='utf-8')
    product = get_product_feature(product)
    print ('get_product_feature finished')
    #调用get_recent_user_feature函数，提取倒数3天、30天，用户ID-行为总数特征
    user_action_feature = get_recent_user_feature(end_date, all_actions)
    print ('get_recent_user_feature finished')
    #调用get_user_productClass_feature函数，提取用户ID-商品类别-行为类别-行为总数特征
    user_productClass_feature = get_user_productClass_feature(start_date, end_date, all_actions)
    print ('get_user_productClass_feature finished')
    #调用get_accumulate_product_feature函数，提取商品类别-行为类别-行为总数特征
    accumulate_product_feature = get_accumulate_product_feature(start_date_30day, end_date, all_actions)
    print ('get_accumulate_product_feature finished')
    #调用get_accumulate_productClass_featur函数，提取商品类别-行为总数、转化率、日均行为总数特征
    accumulate_productClass_feature = get_accumulate_productClass_feature(start_date_30day, end_date, all_actions)
    print ('get_accumulate_productClass_feature finished')
    #调用get_comments_feature函数，提取评论特征
    comment_feature = get_comments_feature(end_date)
    print ('get_comments_feature finished')
    
    actions = None
    #设置7个时间片段的起始时间和终止时间
    for i in (3, 5, 7, 10, 15, 21, 30):
        start_date_30day = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=i)
        start_date_30day = start_date_30day.strftime('%Y-%m-%d')
        if actions is None:
            #调用get_action_feature函数，提取用户ID-商品ID-行为总数特征
            actions = get_action_feature(start_date_30day, end_date, all_actions,i)
        else:
            actions = pd.merge(actions, get_action_feature(start_date_30day, end_date,all_actions,i), how='left',
                              on=['用户ID', '商品ID', '商品类别'])
    #依次连接上述得到的各特征表
    actions = pd.merge(actions, user, how='left', on='用户ID')
    actions = pd.merge(actions, user_action_feature, how='left', on='用户ID')
    actions = pd.merge(actions, user_productClass_feature, how='left', on='用户ID')
    actions = pd.merge(actions, product, how='left', on=['商品ID', '商品类别'])
    actions = pd.merge(actions, accumulate_product_feature, how='left', on='商品ID')
    actions = pd.merge(actions, accumulate_productClass_feature, how='left', on='商品类别')
    actions = pd.merge(actions, comment_feature, how='left', on='商品ID')
    actions = actions.fillna(0)#填充连接后的特征表（actions）的空值
    
    #使用用户真实购买情况作为后续验证
    #设置一个长达5天的时间片段的起始时间和终止时间
    validation_start_date = train_end_date
    validation_end_date = datetime.strptime(validation_start_date, '%Y-%m-%d') + timedelta(days=5)
    validation_end_date = validation_end_date.strftime('%Y-%m-%d')
    #使用make_validation_data函数，获取给定时间片段内购买第8类商品的数据集
    make_validation_data(validation_start_date, validation_end_date, all_actions, 'label_{}'.format(validation_path))
    actions.to_csv('data/result_{}'.format(validation_path), index=False)#将验证集保存

3.10.2.执行make_validation_set函数，构造验证集

In [ ]:
make_validation_set('2016-02-23', '2016-02-26', '验证集.csv', all_actions) 

展示最终得到的验证集

In [ ]:
verification_set=pd.read_csv('data/result_验证集.csv',engine='python',encoding = 'utf-8')#读取最终得到的验证集
verification_set

**3.11.构造测试集**

3.11.1.编写构造测试集的函数make_test_set()

In [ ]:
def make_test_set(start_date, end_date, all_actions):
    #设置长达30天的时间片段的起始时间
    start_date_30day = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=30)
    start_date_30day = start_date_30day.strftime('%Y-%m-%d')
    print('get all actions!')    
    #调用get_user_feature函数，提取用户基本特征
    user = get_user_feature(user_base_info)
    print('get_user_feature finished')
    #调用get_product_feature函数，提取商品基本特征
    product = pd.read_csv(product_path, engine='python', encoding='utf-8')
    product =  get_product_feature(product)
    print('get_product_feature finished')
    #调用get_recent_user_feature函数，提取倒数3天、30天，用户ID-行为总数特征
    user_action_feature = get_recent_user_feature(end_date, all_actions)
    print('get_accumulate_user_feature finished') 
    #调用get_user_productClass_feature函数，提取用户Id-商品类别-行为类别-行为总数特征
    user_productClass_feature = get_user_productClass_feature(start_date, end_date, all_actions)
    print('get_user_productClass_feature finished')
    #调用get_accumulate_product_feature函数，提取商品ID-行为类别-行为总数特征
    accumulate_product_feature = get_accumulate_product_feature(start_date_30day, end_date, all_actions)
    print('get_accumulate_product_feature finished')
    #调用get_accumulate_productClass_feature，提取商品类别-行为总数、转化率、日均行为总数特征
    accumulate_productClass_feature = get_accumulate_productClass_feature(start_date_30day, end_date, all_actions)
    print('get_accumulate_productClass_feat finished') 
    #调用get_comments_feature，提取评论特征
    comment_feature = get_comments_feature(end_date)
    print('get_comments_feature  finished')
    #设置7个时间片段的起始时间和终止时间
    actions = None
    for i in (3, 5, 7, 10, 15, 21, 30):
        start_date_30day = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=i)
        start_date_30day = start_date_30day.strftime('%Y-%m-%d')
        if actions is None:
            #调用get_action_feature，提取用户ID-商品ID-行为总数特征
            actions = get_action_feature(start_date_30day, end_date, all_actions,i)
        else:
            actions = pd.merge(actions, get_action_feature(start_date_30day, end_date,all_actions,i), how='left',
                               on=['用户ID', '商品ID', '商品类别'])
    #依次连接上述得到的各特征表
    actions = pd.merge(actions, user, how='left', on='用户ID')
    actions = pd.merge(actions, user_action_feature , how='left', on='用户ID')
    actions = pd.merge(actions, user_productClass_feature, how='left', on='用户ID')
    actions = pd.merge(actions, product, how='left', on=['商品ID', '商品类别'])
    actions = pd.merge(actions, accumulate_product_feature , how='left', on='商品ID')
    actions = pd.merge(actions, accumulate_productClass_feature, how='left', on='商品类别')
    actions = pd.merge(actions, comment_feature, how='left', on='商品ID')
    
    actions = actions.fillna(0)#填充连接后的特征表（actions）的空值
    actions.to_csv("data/result_测试集.csv", index=False)#将测试集

3.10.2.执行make_test_set函数，构造测试集

In [ ]:
#设置长达3天的时间片段的起始时间和终止时间
test_start_date = '2016-04-13'
test_end_date = datetime.strptime(test_start_date,'%Y-%m-%d')+timedelta(days=3)
test_end_date = test_end_date.strptime('%Y-%m-%d')
#调用make_test_set函数，构造测试集
make_test_set(test_start_date,test_end_date,all_actions)

展示最终得到的测试集

In [ ]:
test_set=pd.read_csv('data/result_测试集.csv',engine='python',encoding = 'utf-8')#读取最终得到的测试集
test_set